In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


In [2]:
data_train = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/tech_data/data_set_train_tech_nonscaled.parquet')

data_test = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/tech_data/data_set_test_tech_nonscaled.parquet')

# firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')

# Top features
# top_features_c = ['cp_flag',	'Ticker',	'date',	'moneyness',	'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond']
# top_features_p = ['cp_flag',	'Ticker',	'date',	'moneyness',	'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock']

# # List of columns to drop
columns_to_drop = ['moneyness_squared_mean', 'moneyness_group', 'tau_squared_mean', 'moneyness_tau_mean', 'tau_squared_std', 'moneyness_tau_std', 'T_std', 'impl_volatility_std', 'moneyness_squared_std']
# # columns_to_drop = ['trading_days_till_exp']


# # Drop columns from datasets if they exist
data_train = data_train.drop(columns=columns_to_drop, errors='ignore')
data_test = data_test.drop(columns=columns_to_drop, errors='ignore')

# Ensure that data_val and data_test have the same column order as data_train
data_test = data_test[data_train.columns]

data_trains = data_train.copy()

# data_train = data_trains[data_trains['date'] < '2020-01-01']
# data_validate = data_trains[data_trains['date'] >= '2020-01-01']

In [3]:
data_train

,date,moneyness_mean,moneyness_std,impl_volatility_mean,T_mean,volume_option_mean,volume_option_std,spread_option_mean,spread_option_std,prc_option_mean,...,cp_flag,FF_rate,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,CLOSE_vix,hi-lo_vix
10,2019-01-02,-0.369385,0.067104,0.487425,0.015873,396.615385,884.822632,1.473846,1.264282,32.380000,...,C,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
11,2019-01-02,-0.138625,0.075746,0.480363,0.015873,1330.062500,2182.422583,0.860000,0.812970,21.559375,...,C,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
12,2019-01-02,0.131133,0.070155,0.484917,0.015873,1891.000000,3115.252592,0.443333,0.362702,16.921000,...,C,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
13,2019-01-02,0.378667,0.074393,0.495387,0.015873,1951.066667,1926.038550,0.336667,0.263213,12.057667,...,C,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
14,2019-01-02,0.639375,0.073574,0.485254,0.015873,2522.500000,5223.991692,0.256875,0.236410,7.835938,...,C,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,2020-12-31,-0.603625,0.085894,0.381434,0.007937,9759.625000,9941.619126,0.236250,0.226775,5.599375,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26
5036,2020-12-31,-0.385857,0.064019,0.391424,0.007937,6169.000000,5507.541073,0.347143,0.384695,8.597857,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26
5037,2020-12-31,-0.142900,0.091652,0.369449,0.007937,11490.100000,17047.601848,0.448000,0.526050,13.340000,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26
5038,2020-12-31,0.119429,0.062543,0.378481,0.007937,11888.571429,17047.082124,0.617143,0.671360,20.870000,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26


In [58]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# # Top features for Call and Put options
top_features_p = ['impl_volatility_mean', 'date', 'cp_flag','prev_iv', 'T_mean', 'CLOSE_vix', 'prev2_iv', 'gold_price', 'prev_iv_std', 'prev2_iv_std', 'reces_indi', 'spread_stock_std', 'moneyness_mean', '5_day_rolling_return_stock_std', 'vol_stock_std', 'hi-lo_stock_std', '5_day_rolling_return_stock_mean', 'RET_std', 'vol_stock_mean', 'NVDA', 'spread_stock_mean', 'hi-lo_vix', 'RET_mean', 'OPEN_vix', 'hi-lo_stock_mean', '1Y_bond', 'AMZN', 'PRC_actual_mean', 'PRC_std', '10Y_RIR', 'prc_option_std', 'ASK_std', 'daily_return_indicator_stock_mean']
top_features_c = ['impl_volatility_mean', 'date', 'cp_flag','prev_iv', 'T_mean', 'gold_price', 'prev_iv_std', 'prev2_iv', 'reces_indi', '5_day_rolling_return_stock_mean', 'OPEN_vix', 'hi-lo_vix', 'prev2_iv_std', 'spread_stock_std', 'vol_stock_mean', '10Y_RIR', 'CLOSE_vix', '5_day_rolling_return_stock_std', '1Y_bond', 'hi-lo_stock_mean', 'RET_mean', 'hi-lo_stock_std', 'PRC_std', 'RET_std', 'FF_rate', 'PRC_actual_mean', 'vol_stock_std', '2Y_bond', 'BID_std', 'moneyness_mean', 'ASK_std', 'spread_stock_mean', 'spread_option_std']
# top_features_c = ['cp_flag', 'Ticker', 'date', 'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond', 'spread_stock', 'spread_vix', 'moneyness','5_day_rolling_return_stock', 'RET','PRC_actual','CLOSE_vix','OPEN_vix','reces_indi' ]
# top_features_c =['prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond', 'spread_stock', 'spread_vix', 'moneyness', '5_day_rolling_return_stock']
# top_features_p =['prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '5_day_rolling_return_stock', 'RET', 'moneyness', 'PRC_actual', 'CLOSE_vix']

# # Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C'][top_features_c]
data_train_p = data_train[data_train['cp_flag'] == 'P'][top_features_p]

# # # Prepare validation data for Call and Put options
# # data_validate_c = data_validate[data_validate['cp_flag'] == 'C'][top_features_c]
# # data_validate_p = data_validate[data_validate['cp_flag'] == 'P'][top_features_p]

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C'][top_features_c]
data_test_p = data_test[data_test['cp_flag'] == 'P'][top_features_p]

In [59]:
data_train_p

,impl_volatility_mean,date,cp_flag,prev_iv,T_mean,CLOSE_vix,prev2_iv,gold_price,prev_iv_std,prev2_iv_std,...,OPEN_vix,hi-lo_stock_mean,1Y_bond,AMZN,PRC_actual_mean,PRC_std,10Y_RIR,prc_option_std,ASK_std,daily_return_indicator_stock_mean
10,0.712199,2019-01-02,P,0.627174,0.015873,28.57,0.000000,449.000000,0.134024,0.000000,...,31.72,33.275610,2.5342,6,63.996469,661.596963,2.66,0.667772,661.955690,0.625000
11,0.655225,2019-01-02,P,0.608392,0.015873,28.57,0.000000,449.000000,0.126452,0.000000,...,31.72,28.612488,2.5342,7,68.752625,656.792705,2.66,0.932805,657.151222,0.500000
12,0.611547,2019-01-02,P,0.606167,0.015873,28.57,0.000000,449.000000,0.138904,0.000000,...,31.72,36.249984,2.5342,6,69.162357,674.869685,2.66,1.306885,675.237459,0.571429
13,0.603932,2019-01-02,P,0.576477,0.015873,28.57,0.000000,449.000000,0.114992,0.000000,...,31.72,36.472126,2.5342,7,68.759143,676.631721,2.66,1.909168,676.995600,0.571429
14,0.578558,2019-01-02,P,0.608202,0.015873,28.57,0.000000,449.000000,0.127452,0.000000,...,31.72,31.537045,2.5342,6,68.067265,658.188818,2.66,2.552109,658.546817,0.529412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,0.381434,2020-12-31,P,0.345824,0.007937,21.31,0.332779,670.825806,0.232085,0.059384,...,21.50,30.738268,0.1105,4,190.049748,1407.987850,0.93,5.543192,1408.212679,0.500000
5036,0.391424,2020-12-31,P,0.314623,0.007937,21.31,0.312908,670.825806,0.174024,0.082468,...,21.50,31.618139,0.1105,3,184.350881,1397.423302,0.93,10.136338,1397.654077,0.647059
5037,0.369449,2020-12-31,P,0.314975,0.007937,21.31,0.299951,670.825806,0.165953,0.060775,...,21.50,32.384067,0.1105,4,185.198813,1399.345646,0.93,16.083932,1399.563337,0.684211
5038,0.378481,2020-12-31,P,0.283760,0.007937,21.31,0.294159,670.825806,0.127616,0.074300,...,21.50,28.011919,0.1105,3,192.844581,1347.749380,0.93,23.985998,1347.967383,0.500000


# All features

In [6]:
# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # # Prepare validation data for Call and Put options
# # data_validate_c = data_validate[data_validate['cp_flag'] == 'C'][top_features_c]
# # data_validate_p = data_validate[data_validate['cp_flag'] == 'P'][top_features_p]

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

In [68]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
import random

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

# Set the seed before training
set_seed(42)
# Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility_mean', 'date', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility_mean']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility_mean', 'date', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility_mean']  # Test target
    
    # # Define the parameter grid for GridSearchCV
    # param_grid = {
    #     'n_estimators': [100, 200],      # Number of trees in the forest
    #     'max_depth': [10, 20, None],     # Maximum depth of the tree
    #     'min_samples_split': [2, 5],     # Minimum number of samples required to split an internal node
    #     'min_samples_leaf': [1, 2],      # Minimum number of samples required to be at a leaf node
    #     'bootstrap': [True, False]       # Whether bootstrap samples are used when building trees
    # }

    # Hyperparameters Best
    # param_grid = {
    #     'max_features': [10,15, 20, 25, 30],
    #     'max_depth': [3,4,5, 6, 7 ,8, 10],
    #     'n_estimators': [50, 100,150, 200,300, 400, 500, 600, 700],
    # } np.linspace(0, 1, 10)

    param_grid = {
    'max_features': [25],
    'n_estimators': [100,150, 200,300],
    'min_samples_leaf': [1, 2, 4, 7],
    'min_samples_split': [2, 5, 10],
    'max_depth': [10, 15, 20]  # Limit depth to avoid overfitting
    }
    # 'max_features': [15],
    # 'n_estimators': [50, 75, 100, 150, 200],
    # 'min_samples_leaf': [1,2, 4],
    # 'min_samples_split': [2, 7, 12],
    # 'max_depth': [10, 20, None]  # Limit depth to avoid overfitting
    # }  

    # param_grid = { 'max_features': [25], 'n_estimators': [300], 'min_samples_leaf': [4], 'min_samples_split': [10], 'max_depth': [20]} #Call

    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=42)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=(param_grid), 
                               cv=3, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best parameters and the best estimator
    best_rf_model = grid_search.best_estimator_
    print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

    # In-sample (training set) predictions
    y_train_pred = best_rf_model.predict(X_train)

    # Out-of-sample (test set) predictions
    y_test_pred = best_rf_model.predict(X_test)

    # Evaluate In-Sample Performance (on Training Data)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nPerformance for {option_type} Options:")
    print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
    print(f"In-Sample R² (Training): {r2_train:.4f}")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


Running GridSearchCV for Call Options...
Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.2s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.2s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.3s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   7.6s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   7.7s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   7.9s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   9.8s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_sample

/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/myenv/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Best Parameters for Call Options: {'max_depth': 20, 'max_features': 25, 'min_samples_leaf': 7, 'min_samples_split': 2, 'n_estimators': 150}

Performance for Call Options:
In-Sample RMSE (Training): 0.0581
In-Sample R² (Training): 0.9394
Out-of-Sample RMSE (Test): 0.1039
Out-of-Sample R² (Test): 0.5040


In [32]:
data_train_p

,date,moneyness_mean,moneyness_std,impl_volatility_mean,T_mean,volume_option_mean,volume_option_std,spread_option_mean,spread_option_std,prc_option_mean,...,cp_flag,FF_rate,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,CLOSE_vix,hi-lo_vix
10,2019-01-02,-1.860643,0.065937,0.712199,0.015873,339.357143,337.590240,0.100000,0.072536,0.915000,...,P,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
11,2019-01-02,-1.623500,0.071893,0.655225,0.015873,392.285714,407.838661,0.135000,0.107256,1.433214,...,P,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
12,2019-01-02,-1.382812,0.078221,0.611547,0.015873,661.625000,1012.993114,0.118750,0.092005,1.699375,...,P,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
13,2019-01-02,-1.116625,0.075100,0.603932,0.015873,1042.750000,1824.079074,0.168750,0.130480,2.609375,...,P,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
14,2019-01-02,-0.872462,0.068266,0.578558,0.015873,480.076923,491.725442,0.225385,0.153711,4.106538,...,P,2.40,449.000000,0.130000,2.66,2.5342,2.5442,31.72,28.57,3.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,2020-12-31,-0.603625,0.085894,0.381434,0.007937,9759.625000,9941.619126,0.236250,0.226775,5.599375,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26
5036,2020-12-31,-0.385857,0.064019,0.391424,0.007937,6169.000000,5507.541073,0.347143,0.384695,8.597857,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26
5037,2020-12-31,-0.142900,0.091652,0.369449,0.007937,11490.100000,17047.601848,0.448000,0.526050,13.340000,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26
5038,2020-12-31,0.119429,0.062543,0.378481,0.007937,11888.571429,17047.082124,0.617143,0.671360,20.870000,...,P,0.09,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.31,1.26


In [69]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
import random

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

# Set the seed before training
set_seed(42)
# Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility_mean', 'date', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility_mean']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility_mean', 'date', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility_mean']  # Test target
    
    # # Define the parameter grid for GridSearchCV
    # param_grid = {
    #     'n_estimators': [100, 200],      # Number of trees in the forest
    #     'max_depth': [10, 20, None],     # Maximum depth of the tree
    #     'min_samples_split': [2, 5],     # Minimum number of samples required to split an internal node
    #     'min_samples_leaf': [1, 2],      # Minimum number of samples required to be at a leaf node
    #     'bootstrap': [True, False]       # Whether bootstrap samples are used when building trees
    # }

    # Hyperparameters Best
    # param_grid = {
    #     'max_features': [10,15, 20, 25, 30],
    #     'max_depth': [3,4,5, 6, 7 ,8, 10],
    #     'n_estimators': [50, 100,150, 200,300, 400, 500, 600, 700],
    # } np.linspace(0, 1, 10)

    param_grid = {
    'max_features': [25],
    'n_estimators': [100,150, 200,300],
    'min_samples_leaf': [1, 2, 4, 7],
    'min_samples_split': [2, 5, 10],
    'max_depth': [10, 15, 20]  # Limit depth to avoid overfitting
    }
    # 'max_features': [15],
    # 'n_estimators': [50, 75, 100],
    # 'min_samples_leaf': [1,2, 4],
    # 'min_samples_split': [2, 7, 12],
    # 'max_depth': [10, 20, None]  # Limit depth to avoid overfitting
    # }  

    # param_grid = { 'max_features': [30], 'n_estimators': [50], 'min_samples_leaf': [10], 'min_samples_split': [2], 'max_depth': [30]} #Put

    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=42)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=(param_grid), 
                               cv=3, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best parameters and the best estimator
    best_rf_model = grid_search.best_estimator_
    print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

    # In-sample (training set) predictions
    y_train_pred = best_rf_model.predict(X_train)

    # Out-of-sample (test set) predictions
    y_test_pred = best_rf_model.predict(X_test)

    # Evaluate In-Sample Performance (on Training Data)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nPerformance for {option_type} Options:")
    print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
    print(f"In-Sample R² (Training): {r2_train:.4f}")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


Running GridSearchCV for Put Options...
Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.5s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.5s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.6s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   6.3s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   6.3s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   6.4s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   8.1s
[CV] END max_depth=10, max_features=25, min_samples_leaf=1, min_samples

/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/myenv/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Best Parameters for Put Options: {'max_depth': 10, 'max_features': 25, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}

Performance for Put Options:
In-Sample RMSE (Training): 0.0607
In-Sample R² (Training): 0.9450
Out-of-Sample RMSE (Test): 0.0946
Out-of-Sample R² (Test): 0.5579


In [60]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import random
import tensorflow as tf

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

# Set the seed before training
set_seed(42)

# Function to calculate Newey-West standard error
def newey_west_standard_error(errors, lag=1):
    """
    Computes the Newey-West standard error for the series of prediction errors.
    
    Parameters:
    - errors: Array-like of prediction errors
    - lag: Maximum lag to use for the Newey-West estimator (default is 1)
    
    Returns:
    - Newey-West standard error of the prediction errors
    """
    X = np.ones(len(errors))  # Constant term for OLS
    ols_model = sm.OLS(errors, X).fit(cov_type='HAC', cov_kwds={'maxlags': lag})
    nw_std_error = ols_model.bse[0]  # Newey-West standard error
    return nw_std_error

# Function to prepare data, train Random Forest using GridSearchCV, and evaluate performance
def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    
    Returns:
    y_test (pd.Series): True values of test data for implied volatility
    y_test_pred (np.array): Predicted values of implied volatility for test data
    nw_std_error (float): Newey-West standard error for test errors
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target

    # Hyperparameter grid
    param_grid = { 'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [4], 'min_samples_split': [2], 'max_depth': [15]} if option_type == 'Put' else \
                 { 'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [1], 'min_samples_split': [3], 'max_depth': [15]}  # Call

    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=42, max_samples=0.8)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                               cv=5, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best estimator
    best_rf_model = grid_search.best_estimator_

    # Out-of-sample (test set) predictions
    y_test_pred = best_rf_model.predict(X_test)

    # Calculate test errors and Newey-West standard error
    errors_test = y_test - y_test_pred
    nw_std_error = newey_west_standard_error(errors_test, lag=1)

    print(f"{option_type} Options Newey-West Standard Error: {nw_std_error:.4f}")

    return y_test, y_test_pred, nw_std_error

# Call the function for Call options data
y_test_c, y_test_pred_c, calls_nw_std_error = prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
y_test_p, y_test_pred_p, puts_nw_std_error = prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')

# Calculate errors for Calls and Puts
calls_errors = y_test_c - y_test_pred_c
puts_errors = y_test_p - y_test_pred_p

# Pad the shorter error list with zeros to ensure both lists have the same length
max_length = max(len(calls_errors), len(puts_errors))

puts_errors_padded = np.pad(puts_errors, (0, max_length - len(puts_errors)), 'constant', constant_values=0)
calls_errors_padded = np.pad(calls_errors, (0, max_length - len(calls_errors)), 'constant', constant_values=0)

# Save the results to a DataFrame and CSV
dm_test_data = pd.DataFrame({
    'Put Errors': puts_errors_padded,
    'Call Errors': calls_errors_padded,
    'Put Newey-West Std Error': [puts_nw_std_error] * max_length,  # Constant value for all rows
    'Call Newey-West Std Error': [calls_nw_std_error] * max_length  # Constant value for all rows
})

# Specify the file path for the CSV
file_path = '/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/performence_evaluation/dm_test_errors_rf.csv'

# Save the DataFrame to the specified path
dm_test_data.to_csv(file_path, index=False)

print(f"Test errors and Newey-West statistics saved to {file_path}")


KeyError: "['impl_volatility', 'Ticker'] not found in axis"

In [ ]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV

# # Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

# def prepare_data_with_gridsearch(train_data, test_data, option_type):
#     """
#     Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
#     Additionally, select the best models with in-sample R² closest to 0.80 and 0.85.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # Define the parameter grid for GridSearchCV
#     param_grid = {
#         'max_features': [15, 20, 25],                 # Use fewer features to reduce model complexity
#         'n_estimators': [100, 200, 300],             # Increase number of trees for better generalization
#         'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
#         'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
#         'max_depth': [15, None]                 # Limit the depth of trees to prevent overfitting
#     }    

#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor()

#     # Initialize GridSearchCV
#     grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
#                                cv=2, n_jobs=-1, verbose=2)

#     # Train the model using GridSearchCV
#     print(f"Running GridSearchCV for {option_type} Options...")
#     grid_search.fit(X_train, y_train)

#     # Initialize tracking variables for the models closest to r² = 0.80 and r² = 0.85
#     closest_to_80 = {'model': None, 'params': None, 'r2_train': float('inf')}
#     closest_to_85 = {'model': None, 'params': None, 'r2_train': float('inf')}

#     # Iterate over the results and select the best models with in-sample R² closest to 0.80 and 0.85
#     for idx, params in enumerate(grid_search.cv_results_['params']):
#         # Get the estimator with the current parameters
#         model = grid_search.estimator.set_params(**params)
#         model.fit(X_train, y_train)

#         # In-sample (training set) predictions
#         y_train_pred = model.predict(X_train)
#         r2_train = r2_score(y_train, y_train_pred)

#         # Track the model closest to r² = 0.80
#         if abs(r2_train - 0.80) < abs(closest_to_80['r2_train'] - 0.80):
#             closest_to_80['model'] = model
#             closest_to_80['params'] = params
#             closest_to_80['r2_train'] = r2_train
        
#         # Track the model closest to r² = 0.85
#         if abs(r2_train - 0.85) < abs(closest_to_85['r2_train'] - 0.85):
#             closest_to_85['model'] = model
#             closest_to_85['params'] = params
#             closest_to_85['r2_train'] = r2_train

#     # Evaluate models closest to r² = 0.80 and r² = 0.85
#     for target_r2, model_info in zip([0.80, 0.85], [closest_to_80, closest_to_85]):
#         if model_info['model'] is None:
#             print(f"\nNo suitable model found for {option_type} Options with in-sample R² closest to {target_r2}.")
#             continue
        
#         # Print the best parameters
#         print(f"\nBest Parameters for {option_type} Options (R² closest to {target_r2}): {model_info['params']}")
#         print(f"In-Sample R² (Training): {model_info['r2_train']:.4f}")

#         # Out-of-sample (test set) predictions
#         y_test_pred = model_info['model'].predict(X_test)

#         # Evaluate Out-of-Sample Performance (on Test Data)
#         rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#         r2_test = r2_score(y_test, y_test_pred)

#         # Print the results
#         print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#         print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')

# print("hello")
